In [0]:
# Colab 相关设置项
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/CLRS/CLRS_notes"

In [0]:
import random

In [0]:
%%capture
%run "第 7 章： 快速排序.ipynb"  # 导入 partition, randomized_partition
%run "第  8 章： 线性时间排序.ipynb" # 导入最大堆的相关内容

## 9.0 相关概念

- **顺序统计量**
  - 第 $i$ 个顺序统计量是指集合中第 $i$ 小的元素
  - $i=1$ 时，为**最小值**
  - $i=n$ 时，为**最大值**
  - 当 $n$ 为奇数时， $i = (n+1)/2$ 为**中位数**
  - 当 $n$ 为奇数时，存在两个中位数，分别位为 $ i = n/2$ 和 $i = n/2 + 1$ 处
  - 不考虑 $n$ 的奇偶性，则中位数总是出现在 $i=\lfloor (n+1)/2 \rfloor$(**下中位数**) 和 $\lceil (n+1)/2 \rceil$ 处(**上中位数**)

- **选择问题**  
  - 寻找第 $i$ 个顺序统计量，可以抽像为选择问题

> 输入：一个包含 $n$ 个（互异的）数的集合 $A$ 和一个整数 $i$, $1 \le i \le n$  
输出： 元素 $x \in i$, 且$A$中恰好有 $i-1$ 个元素小于它 

## 9.1 最大值和最小值

### 找出最小值

- 可通过 $n-1$ 次比较得出

In [0]:
def minimum(A):
  mini = A[0]
  for i in range(1, len(A)):
    if A[i] < mini:
      mini = A[i]
  return mini

In [0]:
A = random.sample(range(100), 10)
print("A = {}".format(A))
print("{} is the minimun item in A".format(minimum(A)))

A = [95, 43, 53, 94, 38, 77, 52, 6, 81, 34]
6 is the minimun item in A


### 找出最大值

- 与最小值一样，也可通过 $n-1$ 次比较得到

In [0]:
def maxmum(A):
  maxm = A[0]
  for i in range(1, len(A)):
    if A[i] > maxm:
      maxm = A[i]
  return maxm

In [0]:
A = random.sample(range(100), 10)
print("A = {}".format(A))
print("{} is the maxmun item in A".format(maxmum(A)))

A = [25, 65, 19, 74, 51, 7, 87, 69, 66, 71]
87 is the maxmun item in A


### 同时找出最大值和最小值

- 通过成对的处理输入，最多只需要 $3\lfloor n/2 \rfloor$ 次比较即可同时找出最大值和最小值
  - 先比较两个输元素，然后将较小的值与当前的最小值进行比较，将较大的值与当前的最大值进行比较

In [0]:
def mini_maxmum(A):
  n = len(A)
  if n % 2 == 0:
    mini, maxm = A[0], A[1]
    start_index = 2
  else:
    mini = maxm = A[0]
    start_index = 1
  for i in range(start_index, len(A), 2):
    if A[i] > A[i+1]:
      tmp_mini, tmp_maxm = A[i+1], A[i]
    else:
      tmp_mini, tmp_maxm = A[i], A[i+1]
    if tmp_mini < mini:
      mini = tmp_mini
    if tmp_maxm > maxm:
      maxm = tmp_maxm
  return mini, maxm
  

In [0]:
A = random.sample(range(100), random.randint(10, 20))
print("A = {}".format(A))
mini, maxm = mini_maxmum(A)
print("{} is the minimun item in A, {} is the maxmun item in A.".format(mini, maxm))

A = [82, 63, 51, 65, 70, 25, 62, 67, 54, 53, 99, 50, 72, 28, 15]
15 is the minimun item in A, 99 is the maxmun item in A.


### 练习

#### 9.1-1 
最坏情况下，需要经过 $n + \lceil lgn \rceil - 2$ 次比较找到 $n$ 个元素中第二小的元素

- 基本思路
  - 借助[锦标赛树](https://blog.csdn.net/lihao161530340/article/details/78938403)，先找出最小值，需要进行 $n-1$ 次比较
  - 第二小的值一定与最小值的比较过，所以与最小值比较过的值中的最小值，即为第二小的值。
  - 叶子节点为 $n$ 的锦标赛树为满二叉树,其高度最高为 $\lceil lgn \rceil $，则最多有 $\lceil lgn \rceil $ 个元素需要进行 $\lceil lgn \rceil -1 $ 次比较
  - 综上分析，在最坏的情况下，需要进行 $n + \lceil lgn \rceil - 2 $次比较

- 代码实现

In [0]:
class NumberWithComparedValue(int):
  """重写 int 类型"""
  def __new__(cls, number):
    return super().__new__(cls, number)

  def __init__(self, number):
    self.compared_values = list()

In [0]:
def find_second_small_item(A):
  A = [NumberWithComparedValue(item) for item in A]
  smallest = find_smallest_item(A, len(A)-1)
  return int(find_smallest_item(smallest.compared_values, len(smallest.compared_values)-1))
  

def find_smallest_item(A, r):
  """找出最小的元素
  @args: A[list]: 其中的元素为 NumberWithComparedValue
      r[int]: 数组最右侧的索引值
  """
  n = r + 1
  if n <=1:
    return A[0]
  mid = n // 2
  for i in range(mid):
    if A[i] > A[i+mid]:
      A[i], A[i+mid] = A[i+mid], A[i]
    A[i].compared_values.append(A[i+mid])
  if n % 2 == 0:
    return find_smallest_item(A, i)
  else:
    A[i+1], A[r] = A[r], A[i+1]
    return find_smallest_item(A, i+1)



In [0]:
A = [4, 3, 2, 1]
find_second_small_item(A)

2

In [0]:
for i in range(500):
  A = [random.randint(1, 1000) for j in range(random.randint(10, 1000))]
  assert(sorted(A)[1] == find_second_small_item(A))

#### 9.1-2  
在最坏情况下，同时找到 $n$ 个元素中的最大值与最小值的比较次数的下界为 $\lceil 3n/2 \rceil - 2$

- 证明：
  1. 对于 $n$ 个元素来说，刚开始共有 $n$ 个数有可能成为 MAX 或 MIN， 设为集合 $\{MAX\}$ 和 $\{MIN\}$
  2. 当 $n$ 个元素成对比较时，每比较一次，则集合 $\{MAX\}$ 和 $\{MIN\}$ 中的可能值都会减 1  
    - 例： $ a \lt b$ 且 $a$ 不可能在集全 $\{MAX\}$ 中，而 $b$ 则不可能在集合 $\{MIN\}$ 中
  3. 2 中成对的比较共有 $\lfloor n/2 \rfloor$ 次，则此时 $\{MAX\}$ 和 $\{MIN\}$ 中还各有 $n - \lfloor n/2 \rfloor = \lceil n/2 \rceil$ 种可能，
  各需要进行 $\lceil n/2 \rceil - 1$ 次比较才能确定最终值
  - 上述几步中所需要比较次数至少为 $$\lfloor n/2 \rfloor + 2\lceil n/2 \rceil - 2 = n + \lceil n/2 \rceil - 2 = \lceil 3n/2 \rceil - 2$$

## 9-2 期望为线性时间的选择算法

### 代码实现

- 选择排序的分治算法 RANDOMIZED-SELECT 是以快速排序算法为基础的
- 快速排序算法会递归的处理 partition 后两边的数组，而 RANDOMIZED-SELECT 只用处理一边的数组

In [0]:
def randomized_select(A, p, r, i):
  """选择数组 A 中在 [p, r] 中的第 k 个顺序统计量"""
  if p == r:
    return A[p]
  q = randomized_partition(A, p, r)
  k = q - p + 1
  if k == i:
    return A[q]
  elif k > i:
    return randomized_select(A, p, q-1, i)
  else:
    return randomized_select(A, q+1, r, i-k)

In [0]:
A = [random.randint(1, 15) for i in range(10)]
print("A = {}".format(A))
i = random.randint(1, 10)
print("A 中的第 {} 个顺序统计量为 {}".format(i, randomized_select(A, 0, len(A)-1, i)))
print("排序后，第 {} 个值为 {}".format(i, sorted(A)[i-1]))

A = [5, 9, 9, 12, 6, 9, 9, 7, 7, 6]
A 中的第 8 个顺序统计量为 9
排序后，第 8 个值为 9


### 期望运行时间的上界

- 定时指标器随机变量：
$$X_k = I\{子数组 \ A[p \cdots q]\} \ 正好包含 k 个元素$$  
  - 假设元素是互异的，有 $$ E[X_k] = 1/n$$
- 根据指示器随机变量的定义，可得下述递归式:  $$T(n) \le \sum_{k=1}^{n}{X_k \times (T(max(k-1, n-k)) + O(n))}$$
- 根据期望的相关性质和代入法，可证得：$$E(T(n)) = O (n)$$

### 练习题

#### 9.2-1  
证明在 RANDOM-SELECT 中，对长度为 0 的数组，不会进行递归调用

- 基本思路： 反证法  
  1. 如果书中伪代码的第 8 行在数组为 $0$ 的情况下发生调用，则 $ p > q - 1$，由此可得 
  $ q - p < 1 \rightarrow k = q - p + 1 < 2 \rightarrow k \le 1 $，要想进入第 8 行，则必须满足 $i < k \le \rightarrow i < 1$, 这与 $ i \ge 1$ 相矛盾  
  2. 如果第 9 行在数组为 $0$ 的情况下发生调用，则 $q + 1 > r \rightarrow k = q - p + 1 > r - p$，而进入第 9 行的前提条件是 $ i > k$，由此可得 $i > k > r - p$, 
  这与 $i \le r - p$ 相矛盾
  - 综合 1，2 可得不会对长度为 $0$ 的数组进行递归调用

#### 9.2-3  
RANDOMIZED-SELECT 的基于循环的版本

In [0]:
def randomized_select_loop(A, i):
  p, r = 0, len(A) - 1
  while p < r:
    q = randomized_partition(A, p, r)
    k = q - p + 1
    if k == i:
      return A[q]
    elif k > i:
      r = q - 1
    else:
      p = q + 1
      i -= k
  return A[p]

In [0]:
A = [random.randint(1, 15) for i in range(10)]
print("A = {}".format(A))
i = random.randint(1, 10)
print("A 中的第 {} 个顺序统计量为 {}".format(i, randomized_select_loop(A, i)))
print("排序后，第 {} 个值为 {}".format(i, sorted(A)[i-1]))

A = [12, 11, 2, 15, 1, 5, 12, 4, 8, 8]
A 中的第 9 个顺序统计量为 12
排序后，第 9 个值为 12


## 9.3 最坏情况下为线性时间的选择算法

### SELECT 算法描述  
  1. 将输入数组的 $n$ 个元素划分为 $\lfloor n/5 \rfloor$ 组，每组 $5$ 个元素，则至多还有一组由剩下的 $n \ mod \ 5 $ 个元素组成
  2. 确定 $\lceil n/5 \rceil$ 组每一组的中位数, 可用插入排序来实现
  3. 对于第 2 步中确认的每组的中位数，递归的调用 SELECT 以找出其下中位数 $x$，
  4. 以 $x$ 为主元，调用 PARTITION，对数组进行分组，设 $x$ 为第 $k$ 小的元素，则有 $n-k$ 个元素在划分的高区
  5. 如果 $i=k$，则返回 $x$, 如果$i < k$, 则在低区递归的调用 SELECT 来找出第 $i$ 小的元素，如果 $i>k$, 则在高区递归的查找第 $i-k$ 小的元素

### 代码实现

In [0]:
def find_median(A, p, r):
  """借助插入排序找出数组 A 指定区域的下中位数"""
  for j in range(p+1, r+1):
    i = j - 1
    key = A[j]
    while i >=p and A[i] > key:
      A[i+1] = A[i]
      i -= 1
    A[i+1] = key
  return A[(p+r)//2]

In [0]:
def select(A, p, r, i):
  if r - p + 1<= 5:
    find_median(A, p, r)
    return A[p + i - 1]
  j = p
  medians = list()
  while True:
    if j+4 <= r:
      medians.append(find_median(A, j, j+4))
      j += 5
    else:
      if j <= r:
        medians.append(find_median(A, j, r)) 
      break
  key = select(medians, 0, len(medians)-1, (len(medians)- 1)//2)
  index = A.index(key)  # 找到 key 对应的 index
  A[index], A[r] = A[r], A[index]
  q = partition(A, p, r)
  k = q - p + 1
  if k == i:
    return A[q]
  elif k > i:
    return select(A, p, q-1, i)
  else:
    return select(A, q+1, r, i-k)

In [0]:
length = random.randint(1000, 2000)
A = [random.randint(1, 10000) for i in range(random.randint(1, length))]
i = random.randint(1, len(A))
print("select 输出的结果: {}".format(select(A, 0, len(A)-1, i)))
print("先排序，再取第 i 个值的结果: {}".format(sorted(A)[i-1])) 

select 输出的结果: 2883
先排序，再取第 i 个值的结果: 2883


### 时间复杂度分析

- 当输入规模为 $n$ 时，设其最坏情况下的运行时间为 $T(n)$
- 第 3 步的运行时间为 $T(\lceil n/5 \rceil)$  
  - 进一步分析，只要第 5 步中递归调用的规模严格的小于 $4n/5$, 即可保证整个算法为线性算法
- 第 4 步划分后，大于 $x$ 的个数至少为: $$3(\lceil{{1 \over 2} \lceil {n \over 5} \rceil} \rceil - 2) \ge {3n \over 10} - 6 $$  
  - 分析过程图示：
    - <img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191226083609.png" width = 300>
  - 则第 5 步中，SELECT 递归调用时，最多作用于 $7n\ /\ 10 + 6$ 个元素
- 经过上述分析，可得： $$T(n) \le T(\lceil n/5 \rceil) + T(7n/10) + 6 + O(n)$$  
  - 通过代入法可得： $T(n) = O(n)$


### 练习

#### 9.3-6
在 $O(nlgk)$ 的时间内，找出某一集合的 $k$ 分位数

- 基本思路  
  - 设输入的数组为 $A$ 且 $d = \lfloor n / k \rfloor$，则求解 $k$ 分位数，相当于求解 $A$ 的第 $ d, 2d, 3d, \cdots 
  (k-1)d$ 个顺序统计量  
  - 如果直接调用 SELECT 进行求解，则时间复杂度为 $O((k-1)n)$  
  - 注意到，每次调用 SELECT 后，会对数组 $A$ 进行一定的左右划分，即
  设查找是第 $i$ 个顺序统计量，运行完 SELECT 后， $A[i-1]$ (数组下标从 0 开始)为所需要顺序量，而当 $j \in [0, i-1)$ 时，
  $A[j] \le A[i]$; 当 $j \in (i-1, len(A))$ 时， $A[j] > A[i]$。  
    - 利用这个性质，则可以利用分治法，减小调用 SELECT 时数组的规模，从而减小时间复杂度  
  - 具体是首先查找中间位置的 $k$ 分位数，然后再分别查找左右数组中间位置的 $k$ 分位数，如此，可近似将数组规模减半。递归调用算法，
  即可查找到所有的 $k$ 分位数。 
- 复杂度分析
  - 不妨设 $n, k$ 均为 $2$ 的整数倍幂，设时间复杂度为 $T(n)$， 则可得：  
    - $$\begin{aligned} T(n) &\le cn + 2T(n/2) \\
                  &\le 2cn + 4T(n/4) \\
                  &\le 3cn + 8T(n/8) \\
                  &\cdots \\
                  &\le cnlgk + kT(n/k) \\&= cnlgk + O(k) \\&= O(nlgk) \\
     \end{aligned}$$
    - 由上式可得 $T(n) = O(nlgk)$

In [0]:
def find_k_quantiles_recursive(A, p, r, d, res):
  """找出数组 A 的的第 k - 1 个分位数"""
  while r - p + 1 >= d:
    middle = ( (r-p + 1)//d + 1)// 2  # 中间分组的个数，最小为 1
    q = p + middle * d - 1  # 在 A 中的索引位置
    index = (q+1) // d  - 1  # 在 res 中的索引位置
    if index < len(res):  # 排除最后一个 k 分数
        res[index] = select(A, p, r, middle * d)
    find_k_quantiles_recursive(A, p, q-1, d, res)
    p = q + 1


def find_k_quantiles(A, k):
  d = len(A) // k
  res = [None] * (k-1)
  find_k_quantiles_recursive(A, 0, len(A)-1, d, res)
  return res

In [0]:
a = random.sample(range(1000), 1000)
find_k_quantiles(a, 10)

[99, 199, 299, 399, 499, 599, 699, 799, 899]

#### 9.3-7  
设计一个 $O(n)$ 时间的算法，对于一个给定的包含 $n$ 个互异元素的集合 $S$ 和正整数 $k \le n$，该算法能够确定集合 $S$ 中最接近于中位数的 $k$ 个整数

- 基本步骤  
  1. 借助 SELECT 找出集合 $S$ 的下中位数，设其为 $x$，即第 $\lfloor (n+1)/2 \rfloor $ 个顺序统计量
  2. 将 $S$ 中的元素分别与 $x$ 相减后取绝对值，得到集合 $B$
  3. 通过 SELECT 算法找出 $B$ 中的第 $(k+1)$ 个顺序统计量，记为 $y$
  4. 遍历数组 $B$,如果其值在 $(0, y]$ 之间，即为所需求解的 $k$ 个整数之一

- 代码实现

In [0]:
def find_k_middle(S, k):
  """找出最接近中位数的 k 个整数，假设 S 中元素互异"""
  middle = select(S, 0, len(S)-1, len(S)//2)
  B = [0] * len(S)
  for index, item in enumerate(S):
    B[index] = abs(item - middle)
  C = B.copy()  # 为了避免调用 SELECT 时打乱 B 中的元素顺序
  max_distance = select(C, 0, len(C)-1, k+1)
  res = [None]*k  # 结果数组
  i = 0
  for index, item in enumerate(B):
    if 0 < item <= max_distance:
      if i >= len(res): # 距离为  max_distance 的值可能有两个，为了避免 res 溢出，任意较小的一个
        break
      res[i] = S[index]
      i += 1
  return res

In [0]:
S  = random.sample(range(100), 100)
k = random.randint(1, 9)
print("k = {}".format(k))
find_k_middle(S, k)

k = 5


[46, 47, 48, 50, 51]

#### 9.3-8  
设 $X[1\cdots n]$ 和 $Y[1\cdots n]$ 为两个有序数组。设计一个算法，能在 $O(lgn)$ 时间内找出 $X$ 和 $Y$ 中 $2n$ 个元素所有
的中位数

- 基本思路  
  1. 设 $i = \lfloor (n+1)/2 \rfloor$，比较 $X[i]$ 和 $Y[i]$ 的大小，如果 $X[i] < Y[i]$，则中位数会在 $X[i \cdots n]$ 和
  $Y[1\cdots i]$ 之间，否则中位数会在 $X[1\cdots i]$ 和 $Y[i\cdots n]$ 之间。如此，则会将问题规模减半
  2. 递归的运行步骤 1， 直至 $X$ 和 $Y$ 中剩余的元素均为1，此时取将小的元素作为中位数
- 复杂度分析  
  - 每次递归，问题规模会减半, 则共会递归调用 $lg(2n) $次。每次递归的时间复杂度为 $O(1)$， 则总的时间复杂度为为 $O(lgn)$

- 代码实现
  - 实际编写代码时，需要注意每次迭代时，两个数组的长度应保持一致

In [0]:
def median_of_two_sorted_array(A, B, a, b, n):
  """ 寻找两个已排序数组的中位数

  @args:
    A[list], B[list]: 已排序的两个数组 
    a[int]: 数组 A 的起始下标 
    b[int]: 数组 B 的起始下标
    n[int]: 当前两个数组的长度
  @return:
    median: 中位数
  """
  if n == 1:
    return A[a] if A[a] < B[b] else B[b]
  if n == 2:
    return find_median([A[a], A[a+1], B[b], B[b+1]], 0, 3)
  median_A = int(a + (n-1) // 2 )
  median_B = int(b + (n-1) // 2)
  if A[median_A] < B[median_B]:
    a = median_A
  else:
    b = median_B
  n = n / 2 + 1 if n % 2 == 0 else (n+1) / 2
  return median_of_two_sorted_array(A, B, a, b, n)

In [0]:
n = random.randint(5, 15)
A = sorted([random.randint(1, 100) for i in range(n)])
B = sorted([random.randint(1, 100) for i in range(n)])
print("A= {} \nB = {}".format(A, B))
print("the median of A and B is: {}".format(median_of_two_sorted_array(A, B, 0, 0, len(B))))
C = A + B
print("The result of the select is: {}".format(select(C, 0, len(C)-1, len(C)//2)))

A= [8, 20, 34, 35, 40, 42, 74, 74, 82, 84, 91, 92, 94] 
B = [1, 4, 9, 19, 20, 32, 43, 45, 50, 56, 65, 79, 83]
the median of A and B is: 43
The result of the select is: 43


#### 9.3-9

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200102201125.png width=600>

- 各直线的总长度只与油井的纵坐标有关，所以可将二维问题转换为一维问题
- 设位置选在 $x$ 处， 各个油井的坐标分别为 $\{x_1, x_2, x_3, \cdots, x_n \}$
- 总的长度 $S$ 为： $$ S = \sum_{x_i < x}{(x-x_i)} + \sum_{x_i \ge x}{(x_i-x)} \quad i \in [1, n]$$
- 除去某些 $x=x_i$ 的点，可得： $${dS \over dx} = \sum_{x_i < x}{1} - \sum_{x_i \ge x}{1} \tag {9.1}$$
- 式 (9.1) 中，随着 $x$ 的增加，左边的值逐渐增大，右边的值逐渐减小，因此 $dS/dx$ 是一个单调递增的函数
- 当 $x < min\{x_1, x_2, \cdots ,x_n\}$时， $dS/dx < 0$，当 $x > max\{x_1, x_2, \cdots, x_n\}$时， $dS/dx > 0$  
  - 说明随着 $x$ 的增大， $S$ 先单调递减，后单调递增
  - 也就是说，当 $dS/dx=0$ 时，$S$ 会取得最小值，即 $$\sum_{x_i < x}{1} = \sum_{x_i \ge x}{1}$$
  - 此时， $x$ 即为集合 $\{x_1, x_2, x_3, \cdots, x_n \}$ 的中位数

## 思考题

### 9-1 有序序列中第 $i$ 个最大的元素  

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200102203547.png width=600>

#### a

- 代码实现

In [0]:
def find_i_max_sort(A, i):
  randomized_quicksort(A, 0, len(A)-1)
  return A[-i:]

In [0]:
A = [random.randint(1, 20) for i in range(10)]
i = random.randint(1, 10)
print("A = {}".format(A))
print("the {} max numbers of A is: {}".format(i, find_i_max_sort(A, i)))

A = [8, 5, 17, 11, 1, 3, 19, 17, 9, 20]
the 8 max numbers of A is: [5, 8, 9, 11, 17, 17, 19, 20]


- 时间复杂度分析  
  - 快速排序时间复杂度为 $O(nlgn)$，遍历 $i$ 个元素的时间复杂度为 $O(i)$
  - 总的时间复杂度为 $O(nlgn+i)$

#### b

- 代码实现

In [0]:
def find_i_max_heap(A, i):
  build_max_heap(A)
  res = [None] * i
  for index in range(i):
    res[i - index - 1] = heap_extract_max(A)
  return res

In [0]:
A = Heap([random.randint(1, 20) for i in range(10)])
i = random.randint(1, 10)
print("A = {}".format(list(A)))
print("the {} max numbers of A is: {}".format(i, find_i_max_heap(A, i)))

A = [12, 11, 7, 10, 18, 14, 16, 14, 7, 19]
the 8 max numbers of A is: [10, 11, 12, 14, 14, 16, 18, 19]


- 复杂度分析  
  - 建堆的时间复杂度为 $O(n)$
  - EXTRACT-MAX 执行一次的时间复杂度为 $O(lgn)$，则执行 $i$ 次的时间复杂度为 $O(ilgn)$
  - 总的时间复杂度为 $O(n + ilgn)$

####c

In [0]:
def find_i_max_select(A, i):
  select(A, 0, len(A)-1, n - i + 1)
  res = A[-i:]
  randomized_quicksort(res, 0, len(res)-1)
  return res

In [0]:
A = [random.randint(1, 20) for i in range(10)]
i = random.randint(1, 10)
print("A = {}".format(A))
print("the {} max numbers of A is: {}".format(i, find_i_max_select(A, i)))

A = [7, 16, 3, 6, 3, 14, 1, 14, 2, 3]
the 6 max numbers of A is: [3, 6, 7, 14, 14, 16]


### 9-2 带权中位数

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200102210632.png width=600>

#### a

- 设 $x_k$ 为带权中位数，则由定义可知： $$\begin{aligned} 
  &count\{x_i < x_k\} * {1\over n } < {1 \over 2} \\
  &count\{x_i > x_k\} * {1\over n } \le {1 \over 2} \\
  \end{aligned}  
  $$
  - 即 $$\begin{aligned} 
  &count\{x_i < x_k\} < {n \over 2} \\
  &count\{x_i > x_k\}  \le {n \over 2} \\
  \end{aligned}  
  $$
  - 由此式可看出， $x_k$ 也为 $x_1, x_2, \cdots, x_n$ 的中位数

#### b

- 基本思路  
  - 先对数组进行排序
  - 然后从小到大，对权重进行累加，当权重第一次大于 $1/2$ 时，则说明此数是带权中位数

In [0]:
class Weighted_Number(float):
  """构造带有权重的类"""
  def __new__(cls, number, weight):
    return super().__new__(cls, number)
  
  def __init__(self, number, weight):
    self.weight = weight

  def __str__(self):
    return "{}--{:.2f}".format(float(self), self.weight)
  
  def __repr__(self):
    return "{}--{:.2f}".format(float(self), self.weight)


In [0]:
def find_weighted_median_by_sort(A):
  randomized_quicksort(A, 0, len(A)-1)
  sum = 0
  for item in A:
    sum += item.weight
    if sum >= 1/2:
      return item

In [0]:
A = []
for item in [0.1, 0.35, 0.05, 0.1, 0.15, 0.05, 0.2]:
  A.append(Weighted_Number(item, item))
find_weighted_median_by_sort(A)

0.2--0.20

In [0]:
length = 10
r = [random.random() for i in range(length)]
s = sum(r)
weight_list = [i/s for i in r]
num_list = [random.randint(1, 100) for i in range(length)]
A = [None] * length
for i in range(length):
  A[i] = Weighted_Number(num_list[i], weight_list[i])
print(A)
print("带权中位数为：{}".format(find_weighted_median_by_sort(A)))
print("排序后的数组A:\n {}".format(sorted(A)))

[15.0--0.10, 39.0--0.05, 61.0--0.04, 75.0--0.10, 8.0--0.09, 78.0--0.05, 35.0--0.19, 86.0--0.14, 82.0--0.03, 30.0--0.19]
带权中位数为：35.0--0.19
排序后的数组A:
 [8.0--0.09, 15.0--0.10, 30.0--0.19, 35.0--0.19, 39.0--0.05, 61.0--0.04, 75.0--0.10, 78.0--0.05, 82.0--0.03, 86.0--0.14]


#### c

- 基本思路
  1. 借助select 算法中相似的方法，找出中位数的中位数 $x$，然后以此为主键，将数组分为左右两部分 
  2. 分别计算 $x$ 左边和右边的加权平均数  
    - 如果两边的加权平均数都小于 $1/2$，则说明 $x$ 即为加权中位数  
    - 如果左边的加权平均数的累加和大于 $1/2$，则说明加权中位数在左边的数组中，对左边的数组进行递归
    - 如果右边的加权平均数的累加和大于 $1/2$，则说明加权中位数在右边的数组中，对右边的数组进行递归
  3. 每进行一次递归调用，可排除一边的数组。需要据此来更新下一次迭代时，左右子数组加权平均数对应的上界

- 代码实现

In [0]:
def find_approximate_median(A, p, r):
  """找出近似的中位数"""
  if r - p + 1 < 5:
    return find_median(A, p, r)
  else:
    medians = []
    i = p
    while True:
      if i + 4 <= r:
        medians.append(find_median(A, i, i+4))
        i = i + 5
      else:
        if i <= r:
          medians.append(find_median(A, i, r))
        break
    return find_approximate_median(medians, 0, len(medians) - 1)


In [0]:
def find_weighted_median_by_select(A, p, r, left, right):
  """"借助与 select 类似的算法求解加权中位数
  
  @args:
    A[list]: 待寻找加权平均数的数组
    p[int]: 数组的左边界
    r[int]: 数组的右边界
    left, right[float]: 左右数组的权重累加值
  """
  approximate_median = find_approximate_median(A, p, r)
  index = p
  while index <= r:
    if A[index] == approximate_median:
      break
    index += 1
  A[index], A[r] = A[r], A[index]
  q = partition(A, p, r)
  sum_left = sum_right = 0
  for i in range(p, q):
    sum_left += A[i].weight
  for i in range(q+1, r+1):
    sum_right += A[i].weight
  if sum_left >= left:
    return find_weighted_median_by_select(A, p, q-1, left, right - sum_right - A[q].weight)
  elif sum_right > right:
    return find_weighted_median_by_select(A, q+1, r, left - sum_left - A[q].weight, right)
  else:
    return A[q]

In [0]:
A = []
for item in [0.1, 0.35, 0.05, 0.1, 0.15, 0.05, 0.2]:
  A.append(Weighted_Number(item, item))
find_weighted_median_by_select(A, 0, len(A)-1, 1/2, 1/2)

0.2--0.20

In [0]:
length = 10
r = [random.random() for i in range(length)]
s = sum(r)
weight_list = [i/s for i in r]
num_list = [random.randint(1, 100) for i in range(length)]
A = [None] * length
for i in range(length):
  A[i] = Weighted_Number(num_list[i], weight_list[i])
print(A)
print("带权中位数为：{}".format(find_weighted_median_by_select(A, 0, len(A)-1, 1/2, 1/2)))
print("排序后的数组A:\n {}".format(sorted(A)))

[73.0--0.02, 11.0--0.10, 14.0--0.02, 46.0--0.18, 70.0--0.05, 82.0--0.05, 82.0--0.17, 10.0--0.11, 20.0--0.20, 22.0--0.10]
带权中位数为：22.0--0.10
排序后的数组A:
 [10.0--0.11, 11.0--0.10, 14.0--0.02, 20.0--0.20, 22.0--0.10, 46.0--0.18, 70.0--0.05, 73.0--0.02, 82.0--0.17, 82.0--0.05]


#### d

- 证明
  - 设邮局的坐标为 $p$, $s = \sum_{i=1}^{n}{\omega_i d(p, p_i)}$  
  - 可得： $$S = \sum_{p_i < p}{\omega_i(p - p_i)} + \sum_{p_i > p}{\omega_i(p_i - p)} $$
  - 除去某此 $p=p_i$ 的点，将 $S$ 对 $p$ 求导，可得： $${dS \over dx} = \sum_{p < p_i}{\omega_i} - \sum_{p>p_i}{\omega_i}$$
    - 易得 $dS/dx$ 是单调递增的
    - 当 $p < min(p_1, p_2, \cdots , p_n)$ 时，$dS / dx < 0$
    - 当 $p > max(p_1, p_2, \cdots , p_n)$ 时，$dS / dx > 0$
  - 由 $dS/dx$ 的性质，可得 $S$ 先递减，后递增，则当 $dS/dx=0$ 时取得最小值
  - 由带权中位数的性质，可得出 $dS/dx$ 符号发生变化，出现在 $x=x_k$ 处，其中 $x_k$ 为带权中位数，也即邮局的最优位置为带权中位数所在的位置

#### e

- 设邮局 Manhattan 距离总和为 $S$, 邮局的坐标点为 $(x, y)$
- 可得：$$S = \sum_{x_i < x}{\omega_i(x - x_i)} + \sum_{x_i > x}{\omega_i(x_i - x)} + 
\sum_{y_i < y}{\omega_i(y - y_i)} + \sum_{y_i > y}{\omega_i(y_i - y)} $$
- 分别将 $S$ 对 $x,y$ 求偏导，即可将问题转化为一维的邮局问题
- 综上，最佳的位置为 $(x_i, y_i)$，其中 $x_i$ 为 $\{x_1, x_2, \cdots, x_n\}$ 的加权平均数， $y_i$ 为 $\{y_1, y_2, \cdots, y_n\}$ 的加权平均数

### 9-3 小顺序统计量

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200104153435.png" width = 800>

#### a

- 基本思路
  - 如果 $i > n / 2$，则直接调用 SELECT 返回结果 （递归的终止条件)，同时返回 $A$ 中的前 $i$ 个元素组成的集合
  - 如果 $i < n/2$，则成对的比较集合中的元素，选择每对中较小的元素组成新的集合进行递归，递归终止时应该返回递归时传入数组的前 $i$ 个最小的元素
    - 可以将成对元素中较小的元素放在数组左边，较大的放在右边
    - 此步的时间复杂度为 $\lfloor n/2 \rfloor + U_i(\lceil n/2 \rceil)$
  - 左边的数组，并不一定包含前 $i$　个最小的元素  
    - 设递归返回的左边数组的前 $i$ 个最小的元素为 $\{y_1, y_2, \cdots , y_n\}$，则需要找到右边数组中与其比较过的另外 $i$ 个元素，设为 $\{ z_1, z_2, \cdots, z_n \}$,然后在此 $2i$
  个元素中借助SELECT寻找第 $i$ 小的元素，同时返回当前数组中前 $i$　个最小的元素

In [0]:


def find_small_order_statistics(A, i):
  A = [NumberWithComparedValue(item) for item in A]
  return find_small_order_statistics_helper(A, len(A)-1, i)[0]

def find_small_order_statistics_helper(A, r, i):
  """以更快的速度查找小顺序统计量"""
  n = r + 1
  if i >= n // 2:
    return select(A, 0, r, i), A[:i]
  for j in range(n//2):
    if A[j] > A[j+n//2]:
      A[j], A[j+n//2] = A[j+n//2], A[j]
    A[j].compared_values.append(A[j+n//2])
  if n % 2 == 0:
    C = find_small_order_statistics_helper(A, j, i)[1]
  else:
    A[j+1], A[r] = A[r], A[j+1]
    A[j+1].compared_values.append(None)
    C = find_small_order_statistics_helper(A, j+1, i)[1]
  for k in range(i):
    item = C[k].compared_values.pop()
    if item is not None:
      C.append(item)
  return select(C, 0, len(C)-1, i), C[:i]
  

In [0]:
length = random.randint(1000, 3000)
A = [random.randint(1, 1000) for item in range(length)]
i = random.randint(1, len(A)//3)
print("数组长度为:{}".format(len(A)))
print("先排序，再取第 {} 个值的结果: {}".format(i, sorted(A)[i-1])) 
print("select 输出的结果: {}".format(find_small_order_statistics(A, i)))

数组长度为:1889
先排序，再取第 517 个值的结果: 289
select 输出的结果: 289


In [0]:
for j in range(100):
  length = random.randint(1000, 3000)
  A = [random.randint(1, 1000) for item in range(length)]
  i = random.randint(1, len(A)//3)
  assert(sorted(A)[i-1] == find_small_order_statistics(A, i))

#### b

- 证明：
  - 可通过代入法证明
  - 设对于较小的 $n$ 和 $i < n / 2 $满足 $$U_i(n) = n + O(T(2i)lg(n/i)) \tag{9.2}$$  
  - 即存在常数 $c$，使得 $$U_i(n) \le n + cT(2i)lg(n/i)$$
    - 如果 $n/4 \le i < n / 2 $，则有 $$\begin{aligned}
    U_i(n) = \lfloor n/2 \rfloor + U_i(\lceil n / 2 \rceil ) + T(2i) \le n/2 + 2T(2i) = n + cT(2i)lg(n/i) + \underbrace{[-n/2 + (2-clg(n/i)T(2i)]}_{x}
    \end{aligned}$$  
      - 取 $c \ge 2$ , 即可保证 $x < 0$，即 $U_i(n) \le n + cT(2i)lg(n/i)$，可得式 (9.2) 成立
    - 如果 $i < n/4$, 则有 $$\begin{aligned}
    U_i(n) &= \lfloor n/2 \rfloor + U_i(\lceil n / 2 \rceil ) + T(2i) \le n + cT(2i)lg(n/(2i)) + T(2i)\\ 
    &= n + cT(2i)lg(n/i) + \underbrace{[(1 + clg(n/(2i)-clg(n/i)]T(2i)}_{x} \\
    &= n + cT(2i)lg(n/i) + \underbrace{(1 - c)T(2i)}_{y}
    \end{aligned}$$  
      - 取 $c \ge 1$， 即可保证 $y\le 0$，即 $U_i(n) \le n + cT(2i)lg(n/i)$，可得式 (9.2) 成立
  - 综上可得，当 $i < n/2$ 时，式 9-2 成立


#### c

如果 $i$ 为常数，则代入式 (9.2) 可得 $U_i(n) = n + O(lgn)$

#### d

代入式 (9.2) 即可求得相应的结果

### 9-4 随机选择的另一种分析方法

<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200105200927.png width=800>

#### a

根据 $k$ 要对于 $i, j$ 的位置，一共有 3 种不同的可能，如下式： $$ E[X_{ijk}] = 
\left\{
  \begin{aligned}
    &{2 \over j - k + 1} \quad\quad &k < i \\
    &{2 \over j - i + 1} &i \le k \le j \\
    &{2 \over k - i + 1} & j < k
  \end{aligned}
\right.
$$

#### b

- 证明：  
  - $$ \begin{aligned}
  E[X_k] = \sum_{i=1}^{n-1}{\sum_{j=i+1}^{n}}{E[X_{ijk}]} &\le 2(\sum_{i=1}^{k}{\sum_{j=k}^{n}}{1 \over j - i + 1} 
  + \sum_{i=k+1}^{n}{\sum_{j=i+1}^{n}}{1 \over j - k + 1} + \sum_{i=1}^{k-2}{\sum_{j=i+1}^{k-1}}{1 \over k - i + 1}) \\
  & \le 2(\underbrace{\sum_{i=1}^{k}{\sum_{j=k}^{n}}{1 \over j - i + 1}}_x
  + \underbrace{{\sum_{j=k+1}^{n}}{j - k - 1 \over j - k + 1}}_y + \underbrace{\sum_{i=1}^{k-2}{k -i - 1\over k - i + 1}}_z)
  \end{aligned} \tag{9.3}$$  
- 证毕


#### c

- 证明：  
  - 设 $x,y,z$ 为式 (9.3) 中的值，可得：
    - $y + z \le n - k + k - 2 = n - 2 $
    - 设 $j - i + 1 = c$，则 $x$ 中等于 $c$ 的 $i,j$ 最多只有 $c$ 个，而且 $c \in [1, n]$，可得：  
      - $x \le c \times 1/c \times n = n$
  - 综上可复 $x+y+z \le 2n - 2 $
  - 可证得 $$E(X_k) \le 4n \tag{9.4}$$
- 证毕

#### d

- 证明:  
  - 设比较的次数为 $X$，则 RANDOMIZED-SELECT 的期望运行时间 $T(n) = n+X$，由式 (9.4) 可得: 
  $$E(T(n)) = E(n+X) = E(n) + E(X) \le 5n$$
  - 可得 $T(n) = O(n)$
- 证毕